In [ ]:
#Algorithm

thresh=['b1':4.6,'b2':4.6,'b3':4.6,'b4':4.6] #initial freezing level
total_position=['b1':0,'b2':0,'b3':0,'b4':0] #total number of level changed so far
total_level=['b1':0,'b2':0,'b3':0,'b4':0] #total sum of level so far: to calculate average
position_average=['b1':0,'b2':0,'b3':0,'b4':0] #level average so far
positive_pos=['b1':0,'b2':0,'b3':0,'b4':0] #total number of positive level so far including zero
negative_pos=['b1':0,'b2':0,'b3':0,'b4':0] #total number of negative level so far
current_balls=['b1','b2','b3','b4'] # current ball
current_level=['b1':0,'b2':0,'b3':0,'b4':0] #current level of every ball in current balls
universal_flag=['b1':False,'b2':False,'b3':False,'b4':False] #universal flag to get green ticket to freeze without much condition

surface_level=-1 #surface level
pair3points=0 #sum of last 3 balls point
pair3count=0 #total number of freezed ball after every 3 balls freezing
pair3pointsTemp=0 #temporary pair3points
points=0 #Net points
balls=60 #Net balls
usedballs=0 #Used balls so far
perballpoint=0.28 #ideal per ball point to get 16 points in 60 balls

#This function will run on every change of position of balls.
def check_at_every_position_change():
    for ball in current_balls:
        #keeping track of total position changed of every ball, new ball will have zero
        total_position[ball]+=1
        #keeping track of total sum of levels of every ball
        total_level[ball]+=current_level[ball]
        #keeping track of average level of every ball
        position_average[ball]=total_level[ball]/total_position[ball]
        
        #keeping track of number of positive and negative levels of every ball
        if current_level[ball]>=0:
            positive_pos[ball]+=1
        else:
            negative_pos[ball]+=1
          
        #checking previous positive and negative levels ratios, if negative is greater than 75%  and total 
        #position so far is greater than 10 then lessen  the threshold to 0.5, not 4.6. It is for those 
        #scanerio when ball hits the surface just after the trigger
        if (total_position[ball]>=10) & (((negative_pos[ball]-positive_pos[ball])/total_position[ball])*100 >= 75) & (negative_pos[ball]>positive_pos[ball]):
            universal_flag[ball]=True
        else:
            universal_flag[ball]=False
            
        #It is learning rate process, specially very useful in those scanerio where ball hits the surface a
        # long time after its triggering. It continuously checks the average of levels so far of every ball.
        #If it is greater than threshold, it learns that probably it will go up more and if it is less than
        #threshold then it further checks if average is greater than zero or less than zero, and decrease the
        #threshold according to current situation of that ball, only if threshold is still greater than zero.
        if position_average[ball]>=thresh[ball]:
            thresh[ball]+=0.001
        else:
            if (thresh[ball]>=0.0) & (position_average[ball]>=0):
                thresh[ball]-=0.001
            if (thresh[ball]>=0.0) & (position_average[ball]<0):
                thresh[ball]-=0.005
        
        #It is the point where it will decide whether to freeze the ball or not, after checking some parameters
        #which is calculated above. Firstly, it checks if current level of ball is greater then updated threshold
        #or not and if it is greater than value 'pair3points+0.28/4'. What is going on in this pair of condition with
        # 'and' operator is: Firstly, it should achieve level greater than threshold and we found out that 
        # idealy we need 0.28 approx point from each ball to get 16 points in 60 balls but we know we cant 
        # achieve 0.28 in every ball so we make it a bit flexible by tracking pair of 3 balls point and 
        # idealy pair of every 3 balls points should be 1 than it will get 16 points in 60 balls. So, if a ball
        # does not get 0.28 in from a ball then it will have to get more points from  next ball. But it
        # was still very tight so we make it further flexible and divide it by 4. In this way, it achieve 
        #1 points in many case from every pair of 3 balls.
        #second condition with or from first condition is for those exceptional scanerio where ball hits the
        #surface just after the trigger, so in second condition it checks if universal flag is true what does 
        #it mean is if most of previous positions of ball is negative or positive in other words it checks if
        #it is going to negative area very fast or not, if it is then it will be true and it will set the threshold
        #to 0.5 so,if 0.5 or more than 0.5 level comes, then it will freeze the ball immediatly
        if ((current_level[ball]>=thresh[ball]) & (current_level[ball]>=(pair3points+0.28)/4)) | ((universal_flag[ball]==True) & (current_level[ball]>=-0.1)):
            #if pair3points target achieved then set new target which will be zero initially.
            if pair3points>0:
                pair3points=0
            
            #freezing process
            points+=current_level[ball]
            points-=0.01
            balls-=1
            usedballs+=1

            #counting the ball out of 3 balls and sum of points from those balls to keep track of pair3points,
            #as pair is of 3 balls
            pair3count+=1
            pair3pointsTemp+=current_level[ball]
          
        #if ball touch the surface then do the freezing process and update pair3points and count as a if player
        #freeze the ball at surface becuase we will get -1 whether it touch the surface or player freeze it on
        #surface.
        if current_level[ball]<=surface_level:
            points+=-1
            points-=0.01
            balls-=1
            usedballs+=1

            pair3count+=1
            pair3pointsTemp+=current_level[ball]
        
        #It is the point where it will decide what is the next target of ball ideally it should be 0.28 every
        #time but we dont get 0.28 from every ball so if a ball get less than that then it increase gradually
        #build pressure on next ball to get more points.
        #Firstly it check if pair of 3 is completed, if it is completed then check what is sum of their points
        #if it is greater than what it should have then it is all ok, like mentioned above the ideal case, but
        #if it is less than target then target of next ball will be increased.
        if pair3count==3:
            pair3points=pair3pointsTemp
            pair3count=0

            if pair3points>=usedballs*perballpoint:
                pair3points=0
            else:
                pair3points=(usedballs*perballpoint)-pair3points
            pair3pointsTemp=0
        
        #check if 16 points achieved in less than or equal to 60 balls then game won otherwise lose. 
        if (points>=16) & (usedball<=60):
            print('YOU WON!!')
            break
        if (usedball>=60) & (points<16):
            print('YOU LOSE!')
            break
            
            